#### This file contains our Second Model 
#### After running our base model, we create/engineer some features to which we think can potentially increase our predictive power, these features include: 
* weather
* route traffic 
* date into datetime modules
* changed categorical variables into dummy variables

##### Note the weather and traffic data was done in different files, please find. Here we import and work on final tables directly 

In [1]:
#import needed libraries & float format definition
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from scipy import stats
from scipy.stats import norm
from matplotlib import pyplot
from matplotlib.pyplot import figure
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error
import time
import datetime

In [2]:
#Loading Ftable
Final_Table_1 = pd.read_csv('Final_Table_1.csv')
Final_Table_1 = Final_Table_1.drop(columns = 'Unnamed: 0')
Final_Table_1 = Final_Table_1.drop(columns = 'dep_delay')

In [3]:
Final_Table_1.head()

,taxi_out,wheels_off,wheels_on,taxi_in,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,...,origin_city_condn_rainny,origin_city_condn_snow,origin_city_condn_sunny,dest_city_condn_cloundy,dest_city_condn_harzard,dest_city_condn_low_rainny,dest_city_condn_low_snow,dest_city_condn_rainny,dest_city_condn_snow,dest_city_condn_sunny
0,12.0,1610.0,1824.0,7.0,1831.0,21.0,0,0,160,153.0,...,0,0,0,0,0,0,0,0,0,1
1,9.0,632.0,729.0,3.0,732.0,-8.0,0,0,75,69.0,...,0,0,0,0,0,0,0,0,0,1
2,8.0,522.0,732.0,8.0,740.0,-15.0,0,0,100,86.0,...,0,0,0,0,0,1,0,0,0,0
3,8.0,1554.0,1825.0,4.0,1829.0,9.0,0,0,290,283.0,...,0,0,0,0,0,0,0,0,0,1
4,6.0,1814.0,1937.0,36.0,2013.0,23.0,0,0,105,125.0,...,0,0,0,0,0,0,0,0,0,1


In [4]:
# Using Stacking Technique 
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [5]:
Sc = StandardScaler()
X = Final_Table_1.drop(columns = 'arr_delay')
y = Final_Table_1['arr_delay']
X = Sc.fit_transform(X)

# # Splitting Data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.80,test_size=0.20, random_state=42)

In [6]:
# setting the base estimators as the base models and final estimator with a linear regression with 5 cross folds.
base_models = [
    ('Linear Regression',LinearRegression()),
    ('SVR',SVR()),
    ('Random Forest',RandomForestRegressor()),
    ]
stacked = StackingRegressor(
    estimators = base_models,
    final_estimator = LinearRegression(),
    cv = 5)

In [7]:
# Base Model 
for name, model in base_models:
    start_time = time.time()
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    end_time = time.time()
    
    r2 = model.score(X_test, y_test)
    print("-------{}-------".format(name))
    print("Coefficient of determination: {}".format(r2))
    print("Computation Time: {}".format(end_time - start_time))
    print("----------------------------------\n")

-------Linear Regression-------
Coefficient of determination: -9.624240960190377e+28
Computation Time: 41.83171796798706
----------------------------------

-------SVR-------
Coefficient of determination: -0.04336010940721047
Computation Time: 687.128271818161
----------------------------------

-------Random Forest-------
Coefficient of determination: 0.24126978907127317
Computation Time: 666.2954411506653
----------------------------------



In [8]:
# Fit the model on training set
import pickle
model = StackingRegressor(
    estimators = base_models,
    final_estimator = LinearRegression(),
    cv = 5)
model.fit(X_train, y_train)
# save the model to disk
filename = 'Model_1.sav'
pickle.dump(model, open(filename, 'wb'))

In [9]:
# Stacked Model 
start_time = time.time()
stacked.fit(X_train, y_train)    
stacked_prediction = stacked.predict(X_test)
end_time = time.time()
stacked_r2 = stacked.score(X_test, y_test)
stacked_rmse = mean_squared_error(y_test, stacked_prediction, squared = False)
print("-------Stacked Ensemble-------")
print("Coefficient of determination: {}".format(stacked_r2))
print("Computation Time: {}".format(end_time - start_time))
print("----------------------------------")

-------Stacked Ensemble-------
Coefficient of determination: 0.23636528237202803
Computation Time: 7635.371336221695
----------------------------------
